# Objectifs

L'objectif de ce notebook est de construire un DataFrame contenant les séquences de transactions (= transactions qui se suivent sans coupure supérieure à 7 jours)


#### Préparation du tableau


In [1]:
import pandas as pd
import numpy as np
import pickle


In [25]:
"""
transactions_by_customer = pd.read_pickle('pickles/transactions_by_customer')
customer_id = transactions_by_customer.at[1011710, 'customer_id']

print(customer_id)

transactions = pd.read_pickle("pickles/transactions_with_quantity")
"""


be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b


In [49]:
# Tri par client / date
transactions = pd.read_pickle("pickles/transactions_with_quantity")
#transactions = transactions[transactions['customer_id'] == "be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b"]
transactions = transactions.sort_values(['customer_id', 't_dat'])
transactions = transactions.reset_index(drop=True)
transactions.head(30)


,customer_id,article_id,t_dat,price,sales_channel_id,quantity
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,2018-09-20,0.030492,2,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,2018-09-20,0.050831,2,1
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,2018-09-20,0.020322,2,1
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,2018-09-20,0.015237,2,1
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,2018-09-20,0.016932,2,1
5,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,2018-09-20,0.016932,2,1
6,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,2018-09-20,0.016932,2,1
7,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,2018-09-20,0.053373,1,1
8,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,2018-09-20,0.045746,2,1
9,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873011,2018-09-20,0.030492,1,1


In [10]:
# Définitions des fonctions et variables nécessaires
max_interval_in_days = 30


def isInSequence(transaction_date_string, previous_transaction_date_string):
    """ Renvoie True si la date de transaction est suffisamment proche de la précédente """
    transaction_date = pd.to_datetime(transaction_date_string)
    previous_transaction_date = pd.to_datetime(
        previous_transaction_date_string)

    delta = transaction_date - previous_transaction_date

    return delta.days <= max_interval_in_days


def addRow(current_transaction):
    """ Ajoute une entrée au DataFrame. """
    global transaction_trains
    return
    new_row = {
        "customer_id": current_transaction['customer_id'],
        "date_start": current_transaction['t_dat'],
        "date_end": current_transaction['t_dat'],
        "train": [current_transaction['article_id']]
    }
    transaction_trains = transaction_trains.append(new_row, ignore_index=True)


def updateRow(current_transaction):
    """ Met à jour la dernière séquence du DataFrame. """
    global transaction_trains
    return

    current_train = transaction_trains.tail(1)

    current_list = current_train['train'].values[0]
    current_list.append(current_transaction['article_id'])

    # Ajout de la transaction au train courant
    transaction_trains.iloc[-1]['train'] = current_list

    # Mise à jour de la date de fin de séquence
    transaction_trains.iloc[-1]['date_end'] = current_transaction['t_dat']


def save():
    global transaction_trains
    global transactions
    global params

    transactions.to_pickle("pickles/transactions_with_train_ids")
    # transaction_trains.to_pickle("pickles/transaction_trains")
    pickle.dump(params, open("pickles/train_iteration_params", 'wb'))


In [50]:
# Intancie un DataFrame pour l'itération
transaction_trains = pd.DataFrame(
    columns=[
        "customer_id",
        "date_start",
        "date_end",
        "train"])
transaction_trains.to_pickle('pickles/transaction_trains')

transactions['train_index'] = 0
transactions.to_pickle('pickles/transactions_with_train_ids')

# Initialiser les paramètres d'itération
params = {
    "current_index": 0,
    "current_customer": "",
    "current_train_index": -1,
    "previous_transaction_date": "1990-02-01"
}

pickle.dump(params, open("pickles/train_iteration_params", 'wb'))


In [15]:
# Boucle sur le tableau des transactions
transactions = pd.read_pickle('pickles/transactions_with_train_ids')

params = pickle.load(open("pickles/train_iteration_params", "rb"))
print(params)

start = params['current_index']
end = np.minimum(transactions.shape[0], start + 10000000)

for i in range(start, end):
    print(f"\r Processing line n°{i}          ", end="")

    current_transaction = transactions.iloc[i]

    if(current_transaction['customer_id'] != params['current_customer']):
        # Création d'une nouvelle séquence au changement de client.
        addRow(current_transaction)
        params['current_train_index'] += 1
        params['current_customer'] = current_transaction['customer_id']

    elif(isInSequence(current_transaction['t_dat'], params['previous_transaction_date'])):
        # Ajout de la transaction à la séquence courante.
        updateRow(current_transaction)
    else:
        # Création d'une nouvelle séquence si la date de la nouvelle
        # transaction est trop éloignée de la précédente.
        addRow(current_transaction)
        params['current_train_index'] += 1

    # Mise à jour de la date de la transaction précédente
    params['previous_transaction_date'] = current_transaction['t_dat']
    transactions.at[i, 'train_index'] = params['current_train_index']

    # Sauvegarder les tableaux tous les 100 000 lignes
    params['current_index'] = i + 1
    if(i != start and i % 200000 == 0):
        print(f"\r Proceed to backup on line n°{i}...", end="")
        save()

# On sauvegarde une fois que c'est fini
print(f"\r Proceed to backup on end                ", end="")
save()


{'current_index': 20000000, 'current_customer': '19d34b63a5c6a1e789d48f6e4d15980ca6da663a89155fa85189f2a083ee3896', 'current_train_index': 6352673, 'previous_transaction_date': '2020-02-07'}
 Proceed to backup on end                

In [2]:
transactions = pd.read_pickle('pickles/transactions_with_train_ids')
transaction_trains = transactions.groupby("train_index", as_index=False).agg(
    date_start=("t_dat", "min"),
    date_end=("t_dat", "max"),
    train=("article_id", lambda x: list(x))
)
transaction_trains.head()


,train_index,date_start,date_end,train
0,0,2018-09-20,2018-09-20,"[541518023, 663713001]"
1,1,2018-09-20,2018-09-20,"[505221001, 505221004, 685687001, 685687003, 6..."
2,2,2018-09-20,2018-09-20,"[501323011, 598859003, 688873011, 688873012, 6..."
3,3,2018-09-20,2018-09-20,"[529841001, 531310002]"
4,4,2018-09-20,2018-09-20,"[377277001, 501820043, 553139001, 631848002, 6..."


In [9]:
transaction_trains.to_pickle('pickles/transaction_trains')


In [2]:
transaction_trains = pd.read_pickle('pickles/transaction_trains')
transaction_trains.tail(20)


,train_index,date_start,date_end,train
9080159,9080159,2020-09-22,2020-09-22,"[706016003, 706016019]"
9080160,9080160,2020-09-22,2020-09-22,"[685813039, 685813043, 935541001]"
9080161,9080161,2020-09-22,2020-09-22,"[761406001, 903420001, 913597001]"
9080162,9080162,2020-09-22,2020-09-22,"[866714016, 922625005]"
9080163,9080163,2020-09-22,2020-09-22,[910048001]
9080164,9080164,2020-09-22,2020-09-22,[919786002]
9080165,9080165,2020-09-22,2020-09-22,"[651337004, 651591001, 789721001, 808699003, 8..."
9080166,9080166,2020-09-22,2020-09-22,"[806388002, 907702003]"
9080167,9080167,2020-09-22,2020-09-22,"[884319003, 901666001]"
9080168,9080168,2020-09-22,2020-09-22,[860833022]


In [36]:
transactions = pd.read_pickle('pickles/transactions_with_train_ids')
transactions.head(20)


,customer_id,article_id,t_dat,price,sales_channel_id,quantity,train_index
0,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,658506001,2018-09-21,0.059305,1,1,0
1,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,662980002,2018-09-21,0.033881,1,1,0
2,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,478549001,2018-09-22,0.050831,2,1,0
3,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,637515003,2018-09-22,0.033881,2,1,0
4,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,667709001,2018-09-22,0.050831,2,1,0
5,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,670655001,2018-09-22,0.067780,2,1,0
6,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,685687001,2018-09-22,0.016932,2,2,0
7,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,554640001,2018-09-26,0.059305,2,1,1
8,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,561500002,2018-09-26,0.015237,2,1,1
9,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,580482002,2018-09-26,0.050831,2,1,1


In [3]:
transaction_trains['length'] = transaction_trains['date_start'].apply(lambda x: pd.to_datetime(
    (x))) - transaction_trains['date_end'].apply(lambda x: pd.to_datetime((x)))


In [5]:
transaction_trains['length_2'] = transaction_trains['length'].apply(
    lambda x: x.days)


In [10]:
transaction_trains = transaction_trains.drop(columns='length_2', axis=1)


KeyError: "['length_2'] not found in axis"

In [11]:
transaction_trains['length'].value_counts()


0    9080179
Name: length, dtype: int64